# Explainability
Generate SHAP plots for all models.

In [ ]:
import pandas as pd
import joblib, shap, matplotlib.pyplot as plt
from pathlib import Path

train_df = pd.read_csv('../datasets/processed_train.csv')
test_df = pd.read_csv('../datasets/processed_test.csv')
X_train = train_df.drop('target', axis=1)
X_test = test_df.drop('target', axis=1)
base = Path('../outputs')
models = {
    'Logistic Regression': joblib.load(base / 'Logistic Regression' / 'model' / 'logistic_regression_model.pkl'),
    'Random Forest': joblib.load(base / 'Random Forest' / 'model' / 'random_forest_model.pkl'),
    'SVM': joblib.load(base / 'SVM' / 'model' / 'svm_model.pkl'),
}
for name, model in models.items():
    explainer = shap.Explainer(model, X_train)
    shap_values = explainer(X_test)
    plot_dir = base / name / 'plots'
    plot_dir.mkdir(parents=True, exist_ok=True)
    shap.summary_plot(shap_values, X_test, show=False)
    plt.savefig(plot_dir / 'shap_summary.png', bbox_inches='tight')
    plt.close()
    shap.plots.waterfall(shap_values[0], show=False)
    plt.savefig(plot_dir / 'shap_local_0.png', bbox_inches='tight')
    plt.close()
